# Monte Carlo methods

This notebook will guide you through a few examples of how the power and speed of the computer processor can shed light on certain statistics related to a wide range of problems.

In [ ]:
# Setup
import numpy as np
import random
import matplotlib.pyplot as plt

## Example 1: Finding area of a region

By randomly selecting points and counting how many are in the region and how many are outside the region, you can get a good estimate of the true area of the region.

We will try to estimate the area of the unit circle. It is all of the points (x,y) such that $x^2+y^2\leq 1$. We will generate random points in the square with corners (1,1),(1,-1),(-1,1),(-1,-1). If it falls within the circle, we will increment ```circle_points```. Then divide ```circle_points``` by ```N=1000```, the total number of points.

In [ ]:
N = 1000
circle_points = 0
for trial in range(N):
    r_x = 2*random.random()-1
    r_y = 2*random.random()-1
    if r_x**2 + r_y**2 <= 1:
        circle_points += 1
print(circle_points/N)

This tells us that $\frac{\text{Area of circle}}{\text{Area of square}}\approx B$ where $B$ is the value you get from the experiment. Since we know that the area of the square is $2^2= 4$, the area of the circle must be $4*B$. Does your answer line up with the geometric definition of the area of a circle?

## Example 2: Findmax

As we have seen in class, the algorithm to find the maximum of a list of $n$ distinct integers is expected to update the maximum $H(n)-1$ times where $H(n)$ is the $n$th Harmonic number, i.e. $H(n) = 1+1/2+1/3+\dots + 1/n$. We will conduct the following experiment: generate a random permutation $q$ of length $n$, then run the findmax algorithm on $q$ and count the number of times the maximum was updated. We will conduct this experiment 1000 times and take the average. Then we will check that against the theoretical expected value.

### Generating random permutations

In order to generate random permutations, I will use the following unranking algorithm

In [ ]:
def random_permutation(n):
    output = []
    sorted_list = [i for i in range(n)]
    for j in range(n):
        r = random.randint(0,n-j-1)
        next_element = sorted_list[r]
        output = output + [next_element]
        sorted_list.remove(next_element)
    return output

You can use the code below to generate a random permutation of length 10 (with integers 0,1,...,9)

In [ ]:
RP = random_permutation(10)
print(RP)

### Findmax function

Here we will write the findmax function that takes a list of $n\geq 1$ integers and returns ```(max_value, num_max_update)``` where ```max_value``` is the value of the maximum and ```num_max_update``` is the number of times the maximum updates 

In [ ]:
def findmax(n_list):
    n = len(n_list)
    num_max_update = 0
    max_value = n_list[0]
    for i in range(1,n):
        if n_list[i] > max_value:
            max_value = n_list[i]
            num_max_update += 1
    return(max_value,num_max_update)

Try it out on some randomly generated inputs. (you can change the value of ```n``` to test other list lengths.)

In [ ]:
n = 10
RP = random_permutation(n)
print(RP)
print(findmax(RP))

### Experiment

Let's run this experiment ```N=10000``` times where ```n=10```. We will total up the number of times max is updated and divide out by ```N``` to estimate the expected value.

In [ ]:
N = 10000
n = 10
total = 0
for j in range(N):
    RP = random_permutation(10)
    FM = findmax(RP)
    total = total + FM[1]
E_exp = total/N
print(E_exp)

### Harmonic numbers

Let's compare our experiment with the theoretical expected value.
Recall that the theoretical expected value for inputs of length $n$ is $H(n)-1$ where $H(n)$ is the $n$th harmonic number.
First let's generate a list of the harmonic numbers from 0 up to $100$.

In [ ]:
H = [0]*101
for i in range(1,101):
    H[i] = H[i-1] + 1/i

Check to see that ```H[10] = 2.928...```,  ```H[50] = 4.499...``` and ```H[100] = 5.187...```

In [ ]:
print(H[10])
print(H[50])
print(H[100])

Find the experimental error by taking the absolute value of the difference and dividing by the theoretical value.

In [ ]:
print(n)
print(E_exp)
print(H[n]-1)
exp_error = abs(E_exp-(H[n]-1))/(H[n]-1)
print(exp_error)

How well does the experiment estimate the theoretical expected value? By making ```N``` larger, the experiment should give better and better estimates.

## Example 3 Collecting a set of baseball cards.

Let's suppose that in each pack of bubble gum you get one baseball card. There are ```n = 10``` different baseball cards and you want to collect them all. What is the expected number of packs of bubble gum you must buy before you collect all 10?

### Experiment

We will initialize an array of 10 0's. Then we will generate a number randomly from 0 to 9. We will change the corresponding 0 to a 1 until the array has all 1's.


In [ ]:
def baseball_cards(n):
    card_array = [0]*n
    packs = 0
    while 0 in card_array:
        r = random.randint(0,n-1)
        card_array[r] = 1
        packs += 1
    return packs

In [ ]:
n = 10
baseball_cards(n)

Let's run the experiment ```N=1000``` times and take the average.

In [ ]:
N = 1000
n = 10
total_packs = 0
for trial in range(N):
    total_packs = total_packs + baseball_cards(n)
print(total_packs/N)

Try this experiment for different values of n. See if you can make a conjecture about the theoretical expected value for an arbitrary n. We will calculate this in class monday.

## Example 4: Random Graphs

There are a few different ways to define a random graph. We will be looking at the Erdős–Rényi model which is a randomly generated undirected graph.

A $G(n,p)$ graph is constructed by starting with $n$ vertices and for every pair of vertices, there is a $p$ probability that there is an edge between the vertices and $1-p$ probability of no edge.

We will be encoding the graph using an adjacency matrix.

There are many fascinating probability questions that you can ask about random graphs. Let's try to experimentally approximate the answer to the questions:


* What is the probability that the graph is connected?
* What is the expected size of the connected component starting from vertex 0?
* What is the expected number of connected components?
* What is the expected maximum size of connected components?
* What is the probability that the graph has an Eulerian Path?

### Generating $G(n,p)$

This function generates a random graph given parameters ```n``` and ```p```.

In [ ]:
def generate_G_n_p(n,p):
    graph = [[0]*n for k in range(n)]     # this generates an n by n matrix of all 0's
    for i in range(n-1):
        for j in range(i+1,n):
            r = random.random()
            if r<p:                      # r is a random number chosen from the interval [0,1]
                graph[i][j] = 1          # since the graph is undirected, you need to assign the same entry
                graph[j][i] = 1          # for (i,j) and (j,i)
    return graph

Try out the function for different values of ```n``` and ```p```

In [ ]:
generate_G_n_p(10,1/5)

### Graphsearch algorithm

This algorithm is a version of graph search we have seen in class. It will use an array ```Status``` to keep track of the sets $X,F,U$. Then ```F``` will be used as a queue where the first element is the next to be considered and the new elements will be added to the end.

In [ ]:
def Graphsearch(G,s):                   # G is an n by n adjacency matrix of a graph and s (starting vertex) is an integer from 0 to n-1
    n = len(G)
    Status = ['U']*n
    Status[s] = ['F']                   # initialize the Status array
    F = [s]                             # F is a queue with only the vertex s
    while len(F)>0:
        w = F[0]
        for i in range(n):
            if G[w][i] == 1:
                if Status[i] == 'U':
                    Status[i] == 'F'
                    F = F + [i]         # add vertex i to the queue F
        Status [w] = 'X'
        F = F[1:]                       # remove vertex w from the queue F
    return Status

Make a random 10 by 10 graph with p = 1/5 and run Graphsearch starting at 0. If ```Status``` is all ```'X'``` then the graph is connected. Otherwise, it is not. Is the graph_1 you generated connected or not?

In [ ]:
graph_1 = generate_G_n_p(10,1/5)
print(np.matrix(graph_1))
Graphsearch(graph_1,0)

### Experiment 1

Let's set ```n=10``` and ```p=1/4```. What is the probability the the graph is connected? We will run this experiment ```N=1000``` times and increment the total everytime the graph is connected. Then average the total.

In [ ]:
N = 1000
n = 10
p = 1/4
total_connected_graphs = 0
for trial in range(N):
    graphnp = generate_G_n_p(n,p)
    GS = Graphsearch(graphnp,0)
    if 'U' not in GS:
        total_connected_graphs += 1
print(total_connected_graphs/N)

What happens when you increase ```n```, does the result change?

What happens when you change ```p```?, how does the result change?

for ```n=10``` let's plot all results for ```p=1/x, 2/x, 3/x, ..., x/x``` for ```x=10```

### Experiment 2

I want to know in general how are ```n```, ```p```, and connectedness all related. For n=10 let's compute all results for p=1/x, 2/x, 3/x, ..., (x-1)/x for x=20. Then plot them using matplotlib. This might take a while to compute for large values of n.

In [ ]:
x = 20
n = 10
N = 1000
data_points = [0]*x
for i in range(1,x):
    total_connected_graphs = 0
    for trial in range(N):
        graphnp = generate_G_n_p(n,i/(x))
        GS = Graphsearch(graphnp,0)
        if 'U' not in GS:
            total_connected_graphs += 1
    data_points[i] = total_connected_graphs/N
print(data_points)

In [ ]:
plt.plot([j/x for j in range(x)],data_points)

You will see a drastic jump from very low probability to very high probability. This is a phenomenon known as a sharp threshold and when n is large, the jump gets more and more steep. This means that for very large graphs, if p is above the threshhold, it is almost certainly connected and if p is below the threshold then it is almost certainly not connected.

### Experiment 3:
what is the expected size of the component that 0 belongs in? We will set ```n=10``` and ```p=1/n=0.1```. To find the size of the component that 0 belongs in, just count the number of ```'X'```'s in the output.

In [ ]:
N = 1000
n = 10
p = 1/n
component_size_total = 0
for trial in range(N):
    graphnp = generate_G_n_p(n,p)
    GS = Graphsearch(graphnp,0)
    component_size = GS.count('X')
    component_size_total = component_size_total + component_size
print(component_size_total/N)

In order to find a relationship between the size of the component containing 0 and G(n,1/n), we can run the experiment for 5,10,15,20,...,100 and plot the results to see if there is a recognizable curve. (This will take several seconds.)

In [ ]:
N = 5000
data_points = [0]*11
for j in range(1,11):
    n = j*10
    p = 1/n
    component_size_total = 0
    for trial in range(N):
        graphnp = generate_G_n_p(n,p)
        GS = Graphsearch(graphnp,0)
        component_size = GS.count('X')
        component_size_total = component_size_total + component_size
    data_points[j] = component_size_total/N

In [ ]:
plt.plot([j*10 for j in range(11)],data_points)

The graph below graphs the same data on a log-log scale. The slope of this graph tells you the exponent of n for the growth rate.

In [ ]:
### %matplotlib qt

In [ ]:
plt.plot([np.log(j*10) for j in range(1,11)],[np.log(k) for k in data_points[1:]])

## Example 5: Colonel Blotto

Suppose I want to know about knight arrangements in colonel blotto. Particularly, I want to know what is the expected maximum number of knights in a single castle. We can randomly choose a number between 0 and ${109\choose 9}-1$ then unrank it into a binary string of length 109 with exactly 9 ones. Then convert it to a knight arrangement and calculate the maximum.

Here is a function to calculate the binomial coefficient:

In [ ]:
def binomial(n,k):
    if k<0:
        return 0
    pr = 1
    for i in range(k):
        pr = pr*(n-i)/(i+1)
    return int(pr)

Here is a recursive form of the unranking algorithm we learned in class:

In [ ]:
def unrank_binomial(n,k,d):
    if n == k:
        return [1]*n
    if k == 0:
        return [0]*n
    if d < binomial(n-1,k):
        return [0] + unrank_binomial(n-1,k,d)
    else:
        return [1] + unrank_binomial(n-1,k-1,d-binomial(n-1,k))

Here is a procedure that converts a fixed density binary string into a knight arrangement sequence of 10 castles:

In [ ]:
def bs2castles(binarystring):
    L = len(binarystring)
    t = sum(binarystring)
    output = [0]*(t+1)
    output_index = 0
    for i in binarystring:
        if i == 0:
            output[output_index] += 1
        else:
            output_index += 1
    return output

Try out the sequence of commands to generate a random knight arrangement

In [ ]:
upper_rank_bound = binomial(109,9)
RI = random.randint(0,upper_rank_bound-1)
print(RI)
UR = unrank_binomial(109,9,RI)
print(UR)
Knight_arrangement = bs2castles(UR)
print(Knight_arrangement)

### Experiment
We will generate ```N = 1000``` knight arrangement and average the total_max_knights.

In [ ]:
N = 1000
total_max_knights = 0
upper_rank_bound = binomial(109,9)
for trial in range(1000):
    RI = random.randint(0,upper_rank_bound-1)
    UR = unrank_binomial(109,9,RI)
    Knight_arrangement = bs2castles(UR)
    max_knights = max(Knight_arrangement)
    total_max_knights = total_max_knights + max_knights
print(total_max_knights/N)